## VGG16-UNet with DL-Kfold (build_vgg16_unet)

In [ ]:
import os
import random
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import backend as K, mixed_precision
from sklearn.model_selection import KFold
from tensorflow.keras.models import Model
from keras_unet_collection.losses import focal_tversky
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.applications import VGG16 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.metrics import MeanIoU
from wandb.integration.keras import WandbCallback

tf.keras.backend.clear_session()
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

import wandb
wandb.init(
    project="VGG"
)
wandb_callback = WandbCallback(
    save_graph=True,
)

def IoU(y_true, y_pred, dtype=tf.float32):
    y_pred = tf.cast(y_pred, dtype)
    y_true = tf.cast(y_true, y_pred.dtype)

    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)
    
    y_true_pos = tf.reshape(y_true, [-1])
    y_pred_pos = tf.reshape(y_pred, [-1])

    area_intersect = tf.reduce_sum(tf.multiply(y_true_pos, y_pred_pos))
    
    area_true = tf.reduce_sum(y_true_pos)
    area_pred = tf.reduce_sum(y_pred_pos)
    area_union = area_true + area_pred - area_intersect
    
    # Return the IoU score
    return tf.math.divide_no_nan(area_intersect, area_union)

def dice_score(y_true, y_pred, smooth=1):
    
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def tversky(y_true, y_pred, smooth=1, alpha=0.7):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)


def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)


def focal_tversky_loss(y_true, y_pred, gamma=0.75):
    tv = tversky(y_true, y_pred)
    return K.pow((1 - tv), gamma)


# Plot sample of model prediction
def plot_sample(X, y, preds, binary_preds, ix=None):
    if ix is None:
        ix = random.randint(0, len(X))

    fig, ax = plt.subplots(1, 4, figsize=(30, 20))
    ax[0].imshow(X[ix, ..., 0], cmap='Greys_r')
    
    ax[0].set_title('US-image', c="white" )
    ax[0].grid(False)

    ax[1].imshow(y[ix].squeeze(), cmap='Greys_r')
    ax[1].set_title('Aponeurosis', c="white")
    ax[1].grid(False)

    ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1, cmap="Greys_r")
    
    ax[2].set_title('Apo-Predicted', c="white")
    ax[2].grid(False)
    
    ax[3].imshow(binary_preds[ix].squeeze(), vmin=0, vmax=0.5, cmap="Greys_r")
    
    ax[3].set_title('Apo-Picture binary', c="white")
    ax[3].grid(False)
    
    plt.savefig(str(ix)+"Pred_area.tif")


def conv_block(inputs, num_filters): # found in model_training
    x = Conv2D(num_filters, 3, padding = "same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, 3, padding = "same")(x) #NOTE Carla was right, this should be x. This is a bug! 
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    return x

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs) #32
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    
    return x

def build_vgg16_unet(input_shape):
    inputs = Input(input_shape)
    
    vgg16 = VGG16(include_top=False, weights="imagenet", input_tensor = inputs)
    #vgg16.summary()
    
    """ Encoder """
    
    # skip connections
    s1 = vgg16.get_layer("block1_conv2").output # 256
    s2 = vgg16.get_layer("block2_conv2").output # 128
    s3 = vgg16.get_layer("block3_conv3").output # 64
    s4 = vgg16.get_layer("block4_conv3").output # 32

    """ Bottleneck/Bridge """
    
    b1 = vgg16.get_layer("block5_conv3").output # 16
    
    """ Decoder """

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    """ Outputs """
    outputs = Conv2D(1, (1, 1), padding = "same", activation="sigmoid")(d4) #binary segmentation
    model = Model(inputs, outputs, name = "VGG16_U-Net")
    return model

# Images will be re-scaled
im_width = 256
im_height = 256
border = 5

image_directory = "../data/analyzed_copy/volume"
mask_directory = "../data/analyzed_copy/mask"  

# list of all images in the path
#print("Total no. of aponeurosis images = ", len(ids))
ids = 0
image_dataset = []

for patient_path in glob.glob(os.path.join(image_directory, "Patient*")):
    
    for mzp_path in glob.glob(os.path.join(patient_path, "MZP*")):
        ids = ids + len(os.listdir(mzp_path))
        for img_path in glob.glob(os.path.join(mzp_path, "*.tif")):
            img = cv2.imread(img_path, 1)
            img = cv2.resize(img, (256,256))
            img = img_to_array(img)
            img = img/255.0
            image_dataset.append(img)  
image_dataset = np.array(image_dataset)

print("Total no. of aponeurosis images = ", ids)
mask_dataset = []

for patient_path in glob.glob(os.path.join(mask_directory, "Patient*")):
    
    for mzp_path in glob.glob(os.path.join(patient_path, "MZP*")):  
        for img_path in glob.glob(os.path.join(mzp_path, "*.tif")):
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (256,256))
            img = img_to_array(img)
            img = img/255.0
            mask_dataset.append(img)        
mask_dataset = np.array(mask_dataset)

# Define some hyperparameters
batch_size = 2
epochs = 100
num_folds = 5

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=False)

#compile the model
VGG16_UNet = build_vgg16_unet((256,256,3)) #input_shape is (256, 256, 3)
model= VGG16_UNet
model.compile(optimizer=Adam(learning_rate=1e-5), loss="binary_crossentropy", metrics= ["accuracy", IoU])

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(image_dataset, mask_dataset):

    callbacks = [
        EarlyStopping(patience=20, verbose=1),
        ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.0000001, verbose=1),
        ModelCheckpoint(f'K-foldno{fold_no}-VGG16-V1-VL-BFRHL-256.keras', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
        CSVLogger(f'K-foldno{fold_no}-VGG16-V1-VL-BFRHL-256.csv', separator=',', append=False)
    ]

    # # Create a generator
    train_datagen = ImageDataGenerator()
    val_datagen = ImageDataGenerator()

    # Create flow from directory or from arrays
    train_generator = train_datagen.flow(image_dataset[train], mask_dataset[train], batch_size=batch_size)
    val_generator = val_datagen.flow(image_dataset[test], mask_dataset[test], batch_size=batch_size)
    

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    results = model.fit(train_generator, batch_size=batch_size, epochs=epochs,
                        callbacks=callbacks, validation_data=val_generator)
    for epoch in range(epochs):
        history = model.fit(
            train_generator,
            batch_size=batch_size,
            epochs=1,  # Train one epoch at a time to log metrics continuously
            callbacks=callbacks,
            validation_data=val_generator
        )
    
    # Extract metrics from history
        logs = {
            "epoch": epoch + 1,
            "train_loss": history.history["loss"][-1],
            "train_accuracy": history.history["accuracy"][-1],
            #"train_iou": history.history["IoU"][-1],
            "val_loss": history.history["val_loss"][-1],
            "val_accuracy": history.history["val_accuracy"][-1],
            "val_iou": history.history["val_IoU"][-1],
            "learning_rate": 1e-5
        }   
        #Log metrics to Wandb
        #wandb.log(logs)
        #wandb.watch(model, log="all", log_freq=10)
    # Increase fold number
    fold_no = fold_no + 1

    tf.keras.backen.clear_session()
#wandb.finish()
fold_no = 1

Total no. of aponeurosis images =  890
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100


/home/dministrator/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


355/356 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8880 - io_u: 0.0968 - loss: 0.3973
Epoch 1: val_loss improved from inf to 0.17054, saving model to K-foldno1-VGG16-V1-VL-BFRHL-256.keras
356/356 ━━━━━━━━━━━━━━━━━━━━ 24s 48ms/step - accuracy: 0.8881 - io_u: 0.0972 - loss: 0.3966 - val_accuracy: 0.9420 - val_io_u: 0.1830 - val_loss: 0.1705 - learning_rate: 1.0000e-05
Epoch 2/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9296 - io_u: 0.3617 - loss: 0.1156
Epoch 2: val_loss improved from 0.17054 to 0.10485, saving model to K-foldno1-VGG16-V1-VL-BFRHL-256.keras
356/356 ━━━━━━━━━━━━━━━━━━━━ 16s 46ms/step - accuracy: 0.9296 - io_u: 0.3618 - loss: 0.1155 - val_accuracy: 0.9486 - val_io_u: 0.2845 - val_loss: 0.1048 - learning_rate: 1.0000e-05
Epoch 3/100
355/356 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9331 - io_u: 0.4689 - loss: 0.0747
Epoch 3: val_loss improved from 0.10485 to 0.08461, saving model to K-foldno1-VGG16-V1-VL-BFRHL-256.keras
356/356 ━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
import keras
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from keras_unet_collection.losses import focal_tversky

# Set up directories
input_directory = 'C:/Users/admin/Documents/DL_Track/Unet_collection/test_images/'  # Directory containing input images
output_directory = 'C:/Users/admin/Documents/DL_Track/Unet_collection/output/images'  # Directory where predicted masks will be saved
# model = keras.models.load_model('C:/Users/admin/Documents/DL_Track/IFSS_Net/IFSS_Net/IFSS_epoch08_FocalTversky_512.h5')
model = keras.models.load_model('C:/Users/admin/Documents/DL_Track/Unet_collection/VGG16-Unet-FocalTversky_epoch100_unaugmented.keras', custom_objects={"focal_tversky": focal_tversky, "IoU": IoU, "dice_score": dice_score})
#model = keras.models.load_model('C:/Users/admin/Documents/DL_Track/Models_DL_Track/Final_models/model-fasc-VGG16-BCE-512.h5', custom_objects={"IoU": IoU})
# Ensure output directory exists
os.makedirs(output_directory, exist_ok=True)

# Function to load and preprocess images from a directory
def load_and_preprocess_image(image_path, target_size=(512, 512)):
    img = load_img(image_path, target_size=target_size)  # Load the image with target size
    img_array = img_to_array(img)  # Convert the image to a numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image to [0, 1] range
    return img_array

# Function to save predicted masks with Pillow
def save_predicted_mask(predicted_mask, image_name):
    # Apply a threshold to make the mask binary (0 or 1)
    binary_mask = (predicted_mask >= 0.05).astype(np.uint8)  # Apply threshold and convert to 0 and 1
    
    # Rescale the binary mask to [0, 255] for saving as an image
    binary_mask = binary_mask * 255
    
    # Convert the numpy array to a Pillow image
    binary_mask_image = Image.fromarray(np.squeeze(binary_mask))
    
    # Save the image using Pillow
    output_path = os.path.join(output_directory, os.path.splitext(image_name)[0] + '.tif')  # Save as .tif
    binary_mask_image.save(output_path)

# Load images, predict masks, and save results
for image_name in os.listdir(input_directory):
    if image_name.endswith('.tif') or image_name.endswith('.png'):  # Change to your file extension
        image_path = os.path.join(input_directory, image_name)
        
        # Load and preprocess the image
        input_image = load_and_preprocess_image(image_path)
        
        # Use the model to predict the mask
        predicted_mask = model.predict(input_image)[0]  # [0] to remove batch dimension
        
        # Save the predicted mask
        save_predicted_mask(predicted_mask, image_name)

print("Predicted masks have been saved to", output_directory)